In [1]:
import import_ipynb
from Read_Data import Read_data
import Neural_Network
from Neural_Network import Trainer
from Neural_Network import DataModule
from Optimiser import optimiser
from Optimiser_ro import optimiser_ro
import numpy as np
import Plots
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from tqdm import tqdm
from sklearn import preprocessing
import math

importing Jupyter notebook from Read_Data.ipynb
importing Jupyter notebook from Plots.ipynb
importing Jupyter notebook from Neural_Network.ipynb
importing Jupyter notebook from Optimiser.ipynb
Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2404093
Academic license - for non-commercial use only - registered to r.zhang-13@student.tudelft.nl
importing Jupyter notebook from Optimiser_ro.ipynb
Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2404093
Academic license - for non-commercial use only - registered to r.zhang-13@student.tudelft.nl


## Parameters

In [2]:
MUL_DEMAND = 1500
MUL_PRICE = 1000/340.2
MUL_YIELD = 2000
TRUE_WEEKS = 12

MAX_EPOCH = 10000
LSTM_NUM = 50

hyper = {'window':          {'Demand': 17,      'Price': 12,     'Yield': 15    },
         'hs':              {'Demand': 24,      'Price': 10,     'Yield': 16    },
         'num_layers':      {'Demand': 1,       'Price': 2,      'Yield': 2     },
         'lr':              {'Demand': 0.00005, 'Price': 0.0001, 'Yield': 0.0001},
         'patience':        {'Demand': 3000,    'Price': 2000,   'Yield': 10000 },
         'warm':            {'Demand': 500,     'Price': 200,    'Yield': 200   },
         'portion_val':     {'Demand': 0.1,     'Price': 0.1,    'Yield': 0.1   },
         'portion_test':    {'Demand': 0.1,     'Price': 0.1,    'Yield': 0.1   },
         'batch_size':      {'Demand': 128,     'Price': 128,    'Yield': 128   }
        }

weight_feedback = 0.0003
bias_feedback = 0

parameters = {'Th': 1,
              'Tt': 2,
              'Zt': 0.1,
              'Zh': 0.1,
              'Ztruck': 100, 
              'theta_i': 0.9,
              'theta_t': 0.9,
              'theta_r': 0.6,
              'weightPrice_more':0.0001,
              'weightPrice_less':-0.0001,
              'weightDiscount':0.5,
              'weightShortage': 2,
              'capacity_inv': 5000,
              'capacity_truck':500,
              'F_proportion': 0.3
              }

## An Ensemble of LSTMs

In [3]:
class Bootstrapping:
    '''
    Using Bootstrapping to train an esemble of LSTMs to get the prediction distribution.
    Only run() should be called, which trains, tests and predicts using an ensemble of LSTMs by calling predict().
    '''
    def __init__(self, read_data: Read_data):
        '''
        read_data: Read_data class.
        '''
        self.LSTM_num = LSTM_NUM
        self.read_data = read_data
        
        self.name = self.read_data.name
        self.predictor = None
        self.predictions = []
        
    def single_LSTM(self) -> list:
        '''
        Initialise and train, test, predict one branch of LSTM in an ensemble.

        return: The prediction results.
        '''
        model = Neural_Network.LSTMNET(input_size=1, hidden_size=hyper['hs'][self.name], num_layers=hyper['num_layers'][self.name], num_classes=1, seq_length=hyper['window'][self.name])
        self.predictor = Trainer(self.name, model, DataModule, self.read_data, max_epoch=MAX_EPOCH, lr=hyper['lr'][self.name], patience=hyper['patience'][self.name], warm=hyper['warm'][self.name]) 
        self.predictor.run(hyper['portion_val'][self.name], hyper['portion_test'][self.name], hyper['batch_size'][self.name])
        self.predictor.predict()
        return self.predictor.prediction_results
    
    def run(self) -> list:
        '''
        Run the whole ensemble of LSTMs.

        return: A list of predictions.
        '''
        with tqdm(total = self.LSTM_num) as pbar:
            for _ in range(self.LSTM_num):
                pbar.update(1)
                self.predictions.append(self.single_LSTM())
        return self.predictions

## Executor

In [4]:
class Scheduler:
    def __init__(self, demand_class: Read_data, price_class: Read_data, yield_class: Read_data):
        self.demand_class = demand_class
        self.price_class = price_class
        self.yield_class = yield_class
        
        self.counter = 0
        self.opt_period = self.demand_class.predict_length
        self.arrangement_record = {}
        self.arrangementRO_record = {}
        self.demand_difference = 0
        self.price_feedback = 0
        self.contract_demand = {}

    def update(self, last_realYield: list, last_realDemand: list, last_realPrice: list, weight_feedback: float=weight_feedback, bias_feedback: float=bias_feedback):
        '''
        Update the market information by calling update() in Read_data. 
        Then calculate the feedback price according to the demand difference.
        Update the counter.
        
        last_realYield, last_realDemand, last_realPrice: The lastest truth information.
        weight...K, B: The weight and bias for modelling the retailers' reaction linearly.
        '''
        self.demand_class.update(last_realDemand)
        self.price_class.update(last_realPrice)
        self.yield_class.update(last_realYield)
        if self.contract_demand != {}:
            self.demand_difference = - sum(last_realDemand) + self.contract_demand[self.counter].sum()
        self.price_feedback = weight_feedback * self.demand_difference + bias_feedback
        self.counter += 1
    
    def predict(self) -> dict:
        '''
        Run the ensembles of yield, price and demand LSTMs.

        return: A dictionary containing lists of predictions of the ensembles.
        '''
        
        self.demand_ensemble = Bootstrapping(self.demand_class)
        self.price_ensemble = Bootstrapping(self.price_class)
        self.yield_ensemble = Bootstrapping(self.yield_class)
        
        print('Training yield LSTM\n')
        yield_predictions = np.array(self.yield_ensemble.run())
        print('Training demand LSTM\n')
        demand_predictions = np.array(self.demand_ensemble.run())
        print('Training price LSTM\n')
        price_predictions = np.array(self.price_ensemble.run()) + self.price_feedback

        return {'demand_prediction': demand_predictions,
                'price_prediction': price_predictions,
                'yield_prediction': yield_predictions
                }
    
    def optimise(self, parameters, yield_prediction, demand_prediction, price_prediction, manual_demand=None, manual_price=None):
        '''
        The optimiser.
        '''
        arrangement = optimiser(parameters, yield_prediction, demand_prediction, price_prediction, self.opt_period, manual_demand, manual_price)
        self.arrangement_record[self.counter] = arrangement
        self.arrangement_record['demand_difference'] = self.demand_difference
        self.arrangement_record['price_feedback'] = self.price_feedback
        self.contract_demand[self.counter] = self.arrangement_record[self.counter]['contract_demand']

    def robust_optimise(self, parameters, yield_prediction, demand_prediction, price_prediction, H_Tao, H_variation, manual_demand=None, manual_price=None):
        '''
        The robust optimiser.
        '''
        arrangement = optimiser_ro(parameters, yield_prediction, demand_prediction, price_prediction, self.opt_period, H_Tao, H_variation, manual_demand, manual_price)
        self.arrangementRO_record[self.counter] = arrangement
        self.arrangementRO_record['demand_difference'] = self.demand_difference
        self.arrangementRO_record['price_feedback'] = self.price_feedback
        self.contract_demand[self.counter] = self.arrangementRO_record[self.counter]['contract_demand']

## Illustration Function

In [5]:
def illustration(arrangement_record, counter):
    '''
    Print the scheduled transportation arrangement.
    '''
    arrangement = arrangement_record[counter]
    print('The yield is: ' + str(arrangement['Y']))
    print('The state of the producer\'s inventory after satisfying the demand is:\n')
    Plots.plot_matrix(arrangement['I'], 1000)
    Plots.plot_lines(arrangement['I'], show_label=0)
    print('The transportation to the retailers is arranged as following images:\n')
    Plots.plot_matrix(arrangement['T'], 1000)
    Plots.plot_lines(arrangement['T'], show_label=0)
    print('The transportation to the retailers in discounted price is arranged as following images:\n')
    Plots.plot_matrix(arrangement['F'], 1000)
    Plots.plot_lines(arrangement['F'], show_label=0)
    
    print('The feedback of the retailers from the next month is:' + str(arrangement['demand_difference']))
    print('The resulted price_adjustment is:' + str(arrangement['price_feedback']))
    print('The weekly contract strawberry transportation amount for the next month is:' + str(arrangement['contract_demand']))
    print('The weekly contract strawberry price for the next month is:' + str(arrangement['contract_price']))
    print('The predicted demand for the next month is:' + str(arrangement['prediction_demand']))
    print('The predicted price for the next month is:' + str(arrangement['prediction_price']))
    print('The real transportation to the retailer in each day is:' + str(arrangement['T_tot'][0]))
    print('The real transportation to the retailer in discounted price in each day is:' + str(arrangement['F_tot']))
    print('The number of trucks arranged to deliver strawberries to the retailers in each day is:' + str(arrangement['truck']))
    print('The expected total profit next month is:' + str(arrangement['Profit']))
    print('The income by delivering the strawberry to the retailers is:' + str(arrangement['Income_retailers']))
    print('The income by delivering the strawberry to the retailers in discounted price is:' + str(arrangement['Income_discount']))
    print('The total transportation cost is:' + str(arrangement['Transportation_cost']))
    print('The cost caused by failing to transport enough strawberry to the retailer is:' + str(arrangement['Shortage_cost']))
    print('The demand difference penalty is: ' + str(arrangement['Difference']))

## Prediction & Save

In [6]:
def predict_all():
    '''
    Predict and save all the predicted results without the feedback.
    '''
    demand = Read_data('Data/Strawberry Demand.csv', 'Demand', true_weeks=TRUE_WEEKS, mul=MUL_DEMAND, window=hyper['window']['Demand'])
    price = Read_data('Data/Strawberry Price.csv', 'Price', true_weeks=TRUE_WEEKS, mul=MUL_PRICE, window=hyper['window']['Price'])
    syield = Read_data('Data/Yield.csv', 'Yield', true_weeks=TRUE_WEEKS, mul=MUL_YIELD, window=hyper['window']['Yield'])
    scheduler = Scheduler(demand, price, syield)

    true_weeks = TRUE_WEEKS
    for i in range(true_weeks//4):
        print('------------ Month ' + str(i) + ' -------------------------')
        scheduler.predict()
        if i == 0:
            yieldEn_predictions = np.array(scheduler.yield_ensemble.predictions)
            priceEn_predictions = np.array(scheduler.price_ensemble.predictions) + scheduler.price_feedback * weight_feedback
            demandEn_predictions = np.array(scheduler.demand_ensemble.predictions)
        else:
            yieldEn_predictions = np.concatenate((yieldEn_predictions, np.array(scheduler.yield_ensemble.predictions)), axis=1)
            priceEn_predictions = np.concatenate((priceEn_predictions, np.array(scheduler.price_ensemble.predictions) + scheduler.price_feedback * weight_feedback), axis=1)
            demandEn_predictions = np.concatenate((demandEn_predictions, np.array(scheduler.demand_ensemble.predictions)), axis=1)

        if -true_weeks + (i+1) * 4 != 0:
            true_demand = demand.true_data[-true_weeks + i * 4: -true_weeks + (i+1) * 4]
            true_price = price.true_data[-true_weeks + i * 4: -true_weeks + (i+1) * 4]
            true_yield = syield.true_data[-true_weeks + i * 4: -true_weeks + (i+1) * 4]
        else:
            true_demand = demand.true_data[-true_weeks + i * 4:]
            true_price = price.true_data[-true_weeks + i * 4:]
            true_yield = syield.true_data[-true_weeks + i * 4:]

        scheduler.update(true_yield, true_demand, true_price)

    np.save('Predictions/yield_prediction', yieldEn_predictions)
    np.save('Predictions/price_prediction', priceEn_predictions)
    np.save('Predictions/demand_prediction', demandEn_predictions)

## Optimise ALL

In [7]:
def optimise_all() -> list:
    '''
    Optimise all the weeks without feedback.

    return: A list containing all the profits.
    '''
    profit_opt = []

    yieldEn_predictions = np.load('Predictions/yield_prediction.npy')
    priceEn_predictions = np.load('Predictions/price_prediction.npy')
    demandEn_predictions = np.load('Predictions/demand_prediction.npy')

    demand = Read_data('Data/Strawberry Demand.csv', 'Demand', true_weeks=TRUE_WEEKS, mul=MUL_DEMAND, window=hyper['window']['Demand'])
    price = Read_data('Data/Strawberry Price.csv', 'Price', true_weeks=TRUE_WEEKS, mul=MUL_PRICE, window=hyper['window']['Price'])
    syield = Read_data('Data/Yield.csv', 'Yield', true_weeks=TRUE_WEEKS, mul=MUL_YIELD, window=hyper['window']['Yield'])

    optimiser = Scheduler(demand, price, syield)

    for i in range(TRUE_WEEKS//4):
        syield = np.mean(yieldEn_predictions[:, i*4:(i+1)*4], axis=0).reshape(-1)   
        demand = np.mean(demandEn_predictions[:, i*4:(i+1)*4], axis=0).reshape(-1)
        price = np.mean(priceEn_predictions[:, i*4:(i+1)*4], axis=0).reshape(-1)

        optimiser.optimise(syield, demand, price)
        profit_opt.append(optimiser.arrangement_record[optimiser.counter]['Profit'])
    
    return profit_opt


In [8]:
# predict_all()
# optimise_all()